# Composite Types: struct 

In this lesson we will learn to create your own composite types also known as structs. We will learn: 

- Differences between abstract and concrete types
- How to define immutable and mutable types 
- How to create a type constructor. 
- Brief intro to multiple dispatch and see how types have a role in it.


## Contents

- [Types as Containers for Data](#Types-as-Containers-for-Data)
- [Defining Types](#Abstract-Types)
- [Multiple Dispatch](#Multiple-Dispatch)
- [Type Constructor](#Type-Constructor)


## Types as Containers for Data

We can think of types as "containers for data" only. Moreover, it is possible to define a type hierarchy so that functions that work for parent type work also for the children (if they are written properly). A parent type can only be an `AbstractType` (like `Number`), while a child can be both an abstract or concrete type (like `Float32`). In the following tree diagram, types in round bubbles are abstract types, while the ones in square bubbles are concrete types.

<img src="../images/types.jpg" alt="Drawing" width="500" height="700"/>


## Defining Types

To declare an `abstract type` we use:

In [ ]:
abstract type Person
end

abstract type Musician <: Person
end

You may find it surprising, but apparently musicians are people, so Musician is a sub-type of Person. There are many kind of musicians, for example rock-stars and classic musicians, so we define two new concrete types (in particular this kind of type is called a composite type):

In [ ]:
mutable struct Rockstar <: Musician
    name::String
    instrument::String
    bandName::String
    headbandColor::String
    instrumentsPlayed::Int
end

struct ClassicMusician <: Musician
    name::String
    instrument::String
end

Notably rock-stars love to change the colour of their headband, so we have made `Rockstar` a `mutable struct`, which is a concrete type whose elements value can be modified. On the contrary, classic musicians are known for their everlasting love for their instrument, which will never change, so we have made `ClassicMusician` an immutable concrete type, which is the default behavior for structs (Rust made a similar design decision).

We can define another sub-type of `Person`, `Physicist`:

In [ ]:
mutable struct Physicist <: Person
    name::String
    sleepHours::Float64
    programmingLanguage::String
end

aurelio = Physicist("Aurelio", 6, "Julia")
println("Name: ", aurelio.name)
println("Sleep hours: ", aurelio.sleepHours)
println("Programming Language: ", aurelio.programmingLanguage)

Since Physicist is a `mutable struct` we can adjust sleep hours:

In [ ]:
aurelio.sleepHours = 8
println("New Sleep hours: ", aurelio.sleepHours)

Unlike a non-mutable struct, in which resetting data would fail:

In [ ]:
aurelio_musician = ClassicMusician("Aurelio", "Violin")
aurelio_musician.instrument = "Guitar"

As you can see, I love violin and I just can’t change my instrument, as ClassicMusician is an `immutable struct`.

I am not a rock-star, but my friend Ricky is one, so we’ll define:

In [ ]:
ricky = Rockstar("Riccardo", "Voice", "Black Lotus", "red", 2)
println(ricky)
println("Ricky's headband color: ", ricky.headbandColor)

## Multiple Dispatch

It is possible to write functions that operate on both abstract and concrete types. For example, every person is likely to have a name, so we can define the following function:

In [ ]:
function introduceMe(person::Person)
    println("Hello, my name is $(person.name).")
end

introduceMe(ricky)
introduceMe(aurelio)

While only musicians play instruments, so we can define the following function:

In [ ]:
function introduceMe(person::Musician)
    println("Hello, my name is $(person.name) and I play $(person.instrument).")
end

introduceMe(ricky)
introduceMe(aurelio)
introduceMe(aurelio_musician)

The `::SomeType` notation indicates to Julia that person has to be of the aforementioned type or a sub-type. Only the most strict type requirement is considered (which is the lowest type in the type tree), for example ricky is a Person, but **more importantly** he is a `Rockstar` (Rockstar is placed lower in the type tree), thus `introduceMe(person::Rockstar)` is called. In other words, the function with the closest type signature will be called.

This is an example of **multiple dispatch**, which means that we have written a single function with different methods depending on the type of the variable. We will come back again to multiple dispatch in this lesson, as it is one of the most important features of Julia and is considered a more advanced topic, together with type annotations. As an anticipation `::Rockstar` is a type annotation, the compiler will check if person is a Rockstar (or a sub-type of it) and if that is true it will execute the function.

- **NOTE 1: since Julia is not object-oriented it favors multiple dispatch + composition**
- **NOTE 2: abstract types do not have "members" or "fields"** see [issue for the design decision](https://github.com/JuliaLang/julia/issues/4935)
- **NOTE 3: concrete types do not have "member" functions, only constructors**

## Type Constructor

When we created the previous types, two constructors were generated automatically (these are called default constructors). One accepts any arguments and calls convert to convert them to the types of the fields, and the other accepts arguments that match the field types exactly (`String` and `String` in the case of `ClassicMusician`). The reason both of these are generated is that this makes it easier to add new definitions without inadvertently replacing a default constructor.

Sometimes it is more convenient to create custom constructor, so that it is possible to assign default values to certain variables, or perform some initial computations.

In [ ]:
mutable struct MyData
    x::Float64
    x2::Float64
    y::Float64
    z::Float64

    # treat them as a new function
    function MyData(x::Float64, y::Float64)
        x2=x^2
        z = sin(x2+y)
        new(x, x2, y, z) # default constructor
    end
    
    function MyData(x::Float64, x2::Float64, y::Float64, z::Float64)
        new(x,x2,y,z)
    end
end

println("Custom constructor: ", MyData(2.0, 3.0) )
println("Default constructor: ", MyData(2.0, 4.0, 3.0, 0.6569865987187891) )

Sometimes it may be useful to use other types for x, x2 and y, so it is possible to use parametric `{T}` types (i.e. types that accept type information at construction time):

In [ ]:
mutable struct MyData2{T<:Real}
    x::T
    x2::T
    y::T
    z::Float64
    function MyData2{T}(x::T, y::T) where {T<:Real}
        x2=x^2
        z = sin(x2+y)
        new(x, x2, y, z)
    end
end


println(MyData2{Float64}(2.0,3.0) )
println(MyData2{Int}(2,3) )

## Conclusions

We have learnt how to define abstract and concrete types, and how to define mutable and immutable structures. We have then learnt how it is possible to define functions that work on custom types and we have introduced multiple dispatch.

## Questions

1. Can we have members in an abstract type?
2. Can we have functions in concrete types?
3. Is multiple dispatch the same as function overload?